In [1]:
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import math
import os

# MNIST
os.makedirs("img", exist_ok=True)


n_epochs = 100
batch_size = 64
lr = 0.0001
img_size = 28
channels = 1
sample_interval = 1000


#pytorch style
img_shape = (channels, img_size, img_size)




In [2]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        
        self.chunk1 = [nn.Linear(128, 128) ,  
                       nn.LeakyReLU(0.2, inplace=True)]
        
        self.chunk2 = [nn.Linear(128, 256) ,  
                       nn.BatchNorm1d(256, 0.8), 
                       nn.LeakyReLU(0.2, inplace=True)]
        
        self.chunk3 = [nn.Linear( 256, 512) ,
                       nn.BatchNorm1d(512, 0.8), 
                       nn.LeakyReLU(0.2, inplace=True) ]
        
        self.chunk4 = [nn.Linear(512 , 1024) ,
                       nn.BatchNorm1d(1024, 0.8), 
                       nn.LeakyReLU(0.2, inplace=True) ]
        
        

        self.model = nn.Sequential(*self.chunk1, *self.chunk2, *self.chunk3, *self.chunk4,
                                    nn.Linear(1024, int(np.prod(img_shape) )), nn.Tanh() )
        
        

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), channels, img_size, img_size)
        return img


    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity
    
    
    
    

# Loss function
adversarial_loss = torch.nn.BCELoss()
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

generator.cuda()
discriminator.cuda()
adversarial_loss.cuda()
    
# download MNIST
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

# SGD optimizer
#optimizer_G = torch.optim.SGD(generator.parameters(), lr=lr, momentum = 0.9  )
#optimizer_D = torch.optim.SGD(discriminator.parameters(), lr=lr, momentum = 0.9 )



optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5 , 0.999))



Tensor = torch.cuda.FloatTensor
    
    





In [ ]:
counter = 0

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        real_imgs = Variable(imgs.type(Tensor))

        optimizer_G.zero_grad()

        # Sample generator noise
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 128))))

        # Generate images
        gen_imgs = generator(z)

        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        #update G's weights to fool disc
        g_loss.backward()
        optimizer_G.step()

        
        optimizer_D.zero_grad()

        #spotting real image
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        
        #spotting fake image
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        
        #combined loss
        d_loss = (real_loss + fake_loss) / 2
        
        #optimize D's weights to discriminate real from fake image
        d_loss.backward()
        optimizer_D.step()

        print(
            "[%d]-[%d] [Gen loss: %f] [Disc loss: %f] "
            % (epoch,  i,   g_loss.item(), d_loss.item())
        )

     #   batches_done = epoch * len(dataloader) + i
        if counter % sample_interval == 0:
            save_image(gen_imgs.data[:30], "img/%d.png" % counter, nrow=5, normalize=True)
            
        counter = counter + 1


[0]-[0] [Gen loss: 0.688379] [Disc loss: 0.686147] 
[0]-[1] [Gen loss: 0.687212] [Disc loss: 0.638071] 
[0]-[2] [Gen loss: 0.686046] [Disc loss: 0.594652] 
[0]-[3] [Gen loss: 0.684707] [Disc loss: 0.557533] 
[0]-[4] [Gen loss: 0.683381] [Disc loss: 0.525877] 
[0]-[5] [Gen loss: 0.682101] [Disc loss: 0.495910] 
[0]-[6] [Gen loss: 0.680390] [Disc loss: 0.469672] 
[0]-[7] [Gen loss: 0.678658] [Disc loss: 0.449584] 
[0]-[8] [Gen loss: 0.676415] [Disc loss: 0.432822] 
[0]-[9] [Gen loss: 0.674432] [Disc loss: 0.419959] 
[0]-[10] [Gen loss: 0.671803] [Disc loss: 0.409721] 
[0]-[11] [Gen loss: 0.668708] [Disc loss: 0.401188] 
[0]-[12] [Gen loss: 0.665047] [Disc loss: 0.398546] 
[0]-[13] [Gen loss: 0.663568] [Disc loss: 0.391954] 
[0]-[14] [Gen loss: 0.659068] [Disc loss: 0.389576] 
[0]-[15] [Gen loss: 0.654360] [Disc loss: 0.389344] 
[0]-[16] [Gen loss: 0.649374] [Disc loss: 0.387271] 
[0]-[17] [Gen loss: 0.645582] [Disc loss: 0.389385] 
[0]-[18] [Gen loss: 0.639323] [Disc loss: 0.389283] 
[0]

[0]-[163] [Gen loss: 0.700936] [Disc loss: 0.510418] 
[0]-[164] [Gen loss: 0.763702] [Disc loss: 0.515819] 
[0]-[165] [Gen loss: 0.871102] [Disc loss: 0.482774] 
[0]-[166] [Gen loss: 0.700579] [Disc loss: 0.496891] 
[0]-[167] [Gen loss: 0.825485] [Disc loss: 0.497771] 
[0]-[168] [Gen loss: 0.749384] [Disc loss: 0.500276] 
[0]-[169] [Gen loss: 0.776044] [Disc loss: 0.510432] 
[0]-[170] [Gen loss: 0.766937] [Disc loss: 0.520224] 
[0]-[171] [Gen loss: 0.714024] [Disc loss: 0.509175] 
[0]-[172] [Gen loss: 0.776732] [Disc loss: 0.508804] 
[0]-[173] [Gen loss: 0.723967] [Disc loss: 0.534928] 
[0]-[174] [Gen loss: 0.731405] [Disc loss: 0.532215] 
[0]-[175] [Gen loss: 0.699435] [Disc loss: 0.500758] 
[0]-[176] [Gen loss: 0.789526] [Disc loss: 0.517415] 
[0]-[177] [Gen loss: 0.738910] [Disc loss: 0.488008] 
[0]-[178] [Gen loss: 0.821391] [Disc loss: 0.486449] 
[0]-[179] [Gen loss: 0.753209] [Disc loss: 0.481679] 
[0]-[180] [Gen loss: 0.831278] [Disc loss: 0.460180] 
[0]-[181] [Gen loss: 0.82026

[0]-[326] [Gen loss: 0.822872] [Disc loss: 0.509645] 
[0]-[327] [Gen loss: 0.826388] [Disc loss: 0.522564] 
[0]-[328] [Gen loss: 0.778554] [Disc loss: 0.534326] 
[0]-[329] [Gen loss: 0.762547] [Disc loss: 0.558072] 
[0]-[330] [Gen loss: 0.685360] [Disc loss: 0.567995] 
[0]-[331] [Gen loss: 0.909132] [Disc loss: 0.573252] 
[0]-[332] [Gen loss: 0.533293] [Disc loss: 0.609610] 
[0]-[333] [Gen loss: 0.977258] [Disc loss: 0.606624] 
[0]-[334] [Gen loss: 0.485332] [Disc loss: 0.648477] 
[0]-[335] [Gen loss: 0.874002] [Disc loss: 0.614896] 
[0]-[336] [Gen loss: 0.611296] [Disc loss: 0.607750] 
[0]-[337] [Gen loss: 0.808319] [Disc loss: 0.619133] 
[0]-[338] [Gen loss: 0.535061] [Disc loss: 0.631871] 
[0]-[339] [Gen loss: 0.956988] [Disc loss: 0.609773] 
[0]-[340] [Gen loss: 0.479438] [Disc loss: 0.633104] 
[0]-[341] [Gen loss: 0.994028] [Disc loss: 0.565887] 
[0]-[342] [Gen loss: 0.659484] [Disc loss: 0.555089] 
[0]-[343] [Gen loss: 0.771092] [Disc loss: 0.531430] 
[0]-[344] [Gen loss: 0.87538

[0]-[479] [Gen loss: 0.883816] [Disc loss: 0.412947] 
[0]-[480] [Gen loss: 1.094510] [Disc loss: 0.448142] 
[0]-[481] [Gen loss: 0.671089] [Disc loss: 0.451514] 
[0]-[482] [Gen loss: 1.377491] [Disc loss: 0.483365] 
[0]-[483] [Gen loss: 0.575726] [Disc loss: 0.501632] 
[0]-[484] [Gen loss: 1.160380] [Disc loss: 0.391140] 
[0]-[485] [Gen loss: 1.054691] [Disc loss: 0.371296] 
[0]-[486] [Gen loss: 0.916766] [Disc loss: 0.392154] 
[0]-[487] [Gen loss: 1.089774] [Disc loss: 0.383736] 
[0]-[488] [Gen loss: 0.951635] [Disc loss: 0.378224] 
[0]-[489] [Gen loss: 1.054879] [Disc loss: 0.399081] 
[0]-[490] [Gen loss: 0.883895] [Disc loss: 0.424124] 
[0]-[491] [Gen loss: 0.922914] [Disc loss: 0.427841] 
[0]-[492] [Gen loss: 0.953309] [Disc loss: 0.429234] 
[0]-[493] [Gen loss: 0.884706] [Disc loss: 0.425956] 
[0]-[494] [Gen loss: 1.005674] [Disc loss: 0.429555] 
[0]-[495] [Gen loss: 0.848499] [Disc loss: 0.427839] 
[0]-[496] [Gen loss: 1.103025] [Disc loss: 0.442806] 
[0]-[497] [Gen loss: 0.70864

[0]-[642] [Gen loss: 1.299639] [Disc loss: 0.391239] 
[0]-[643] [Gen loss: 0.869938] [Disc loss: 0.390339] 
[0]-[644] [Gen loss: 1.163586] [Disc loss: 0.391209] 
[0]-[645] [Gen loss: 0.941354] [Disc loss: 0.403799] 
[0]-[646] [Gen loss: 1.076296] [Disc loss: 0.369812] 
[0]-[647] [Gen loss: 1.052395] [Disc loss: 0.382825] 
[0]-[648] [Gen loss: 1.149041] [Disc loss: 0.389037] 
[0]-[649] [Gen loss: 0.871446] [Disc loss: 0.421834] 
[0]-[650] [Gen loss: 1.194578] [Disc loss: 0.399172] 
[0]-[651] [Gen loss: 0.888437] [Disc loss: 0.422020] 
[0]-[652] [Gen loss: 1.118425] [Disc loss: 0.439331] 
[0]-[653] [Gen loss: 0.814371] [Disc loss: 0.382612] 
[0]-[654] [Gen loss: 1.559561] [Disc loss: 0.403561] 
[0]-[655] [Gen loss: 0.694830] [Disc loss: 0.429044] 
[0]-[656] [Gen loss: 1.524027] [Disc loss: 0.393382] 
[0]-[657] [Gen loss: 0.829197] [Disc loss: 0.390956] 
[0]-[658] [Gen loss: 1.222628] [Disc loss: 0.356268] 
[0]-[659] [Gen loss: 1.108347] [Disc loss: 0.361796] 
[0]-[660] [Gen loss: 1.02893

...

[4]-[499] [Gen loss: 0.896152] [Disc loss: 0.329456] 
[4]-[500] [Gen loss: 2.403569] [Disc loss: 0.433125] 
[4]-[501] [Gen loss: 0.704121] [Disc loss: 0.417185] 
[4]-[502] [Gen loss: 2.520640] [Disc loss: 0.314286] 
[4]-[503] [Gen loss: 1.391225] [Disc loss: 0.254360] 
[4]-[504] [Gen loss: 1.431603] [Disc loss: 0.272273] 
[4]-[505] [Gen loss: 1.681750] [Disc loss: 0.303373] 
[4]-[506] [Gen loss: 1.174171] [Disc loss: 0.296232] 
[4]-[507] [Gen loss: 1.877769] [Disc loss: 0.311131] 
[4]-[508] [Gen loss: 1.225504] [Disc loss: 0.353182] 
[4]-[509] [Gen loss: 1.369160] [Disc loss: 0.271058] 
[4]-[510] [Gen loss: 1.781041] [Disc loss: 0.280393] 
[4]-[511] [Gen loss: 1.271935] [Disc loss: 0.347057] 
[4]-[512] [Gen loss: 1.273855] [Disc loss: 0.339840] 
[4]-[513] [Gen loss: 1.742297] [Disc loss: 0.290466] 
[4]-[514] [Gen loss: 1.275165] [Disc loss: 0.326297] 
[4]-[515] [Gen loss: 1.436038] [Disc loss: 0.322370] 
[4]-[516] [Gen loss: 1.438900] [Disc loss: 0.355210] 
[4]-[517] [Gen loss: 1.02944

[4]-[654] [Gen loss: 0.543573] [Disc loss: 0.543894] 
[4]-[655] [Gen loss: 3.117923] [Disc loss: 0.814815] 
[4]-[656] [Gen loss: 0.356930] [Disc loss: 0.724476] 
[4]-[657] [Gen loss: 2.638303] [Disc loss: 0.536508] 
[4]-[658] [Gen loss: 0.844669] [Disc loss: 0.425170] 
[4]-[659] [Gen loss: 1.556863] [Disc loss: 0.384987] 
[4]-[660] [Gen loss: 1.180732] [Disc loss: 0.354426] 
[4]-[661] [Gen loss: 1.560546] [Disc loss: 0.393355] 
[4]-[662] [Gen loss: 1.013977] [Disc loss: 0.424214] 
[4]-[663] [Gen loss: 1.498041] [Disc loss: 0.408539] 
[4]-[664] [Gen loss: 1.115803] [Disc loss: 0.451795] 
[4]-[665] [Gen loss: 1.032159] [Disc loss: 0.409104] 
[4]-[666] [Gen loss: 2.098291] [Disc loss: 0.532460] 
[4]-[667] [Gen loss: 0.406554] [Disc loss: 0.683972] 
[4]-[668] [Gen loss: 2.883092] [Disc loss: 0.772740] 
[4]-[669] [Gen loss: 0.310812] [Disc loss: 0.760094] 
[4]-[670] [Gen loss: 2.956856] [Disc loss: 0.594995] 
[4]-[671] [Gen loss: 0.946415] [Disc loss: 0.419092] 
[4]-[672] [Gen loss: 1.49417

[4]-[809] [Gen loss: 2.592703] [Disc loss: 0.479047] 
[4]-[810] [Gen loss: 0.768650] [Disc loss: 0.433075] 
[4]-[811] [Gen loss: 2.114217] [Disc loss: 0.312586] 
[4]-[812] [Gen loss: 1.420317] [Disc loss: 0.361296] 
[4]-[813] [Gen loss: 0.889298] [Disc loss: 0.442598] 
[4]-[814] [Gen loss: 2.179365] [Disc loss: 0.409506] 
[4]-[815] [Gen loss: 0.831987] [Disc loss: 0.453726] 
[4]-[816] [Gen loss: 1.954630] [Disc loss: 0.348230] 
[4]-[817] [Gen loss: 1.136804] [Disc loss: 0.324059] 
[4]-[818] [Gen loss: 1.880033] [Disc loss: 0.254571] 
[4]-[819] [Gen loss: 1.514431] [Disc loss: 0.328030] 
[4]-[820] [Gen loss: 1.089028] [Disc loss: 0.352522] 
[4]-[821] [Gen loss: 2.168771] [Disc loss: 0.354235] 
[4]-[822] [Gen loss: 1.362851] [Disc loss: 0.341232] 
[4]-[823] [Gen loss: 1.357741] [Disc loss: 0.328325] 
[4]-[824] [Gen loss: 1.720884] [Disc loss: 0.330778] 
[4]-[825] [Gen loss: 1.221539] [Disc loss: 0.357946] 
[4]-[826] [Gen loss: 1.639800] [Disc loss: 0.403698] 
[4]-[827] [Gen loss: 0.86946

[5]-[28] [Gen loss: 2.841830] [Disc loss: 0.367879] 
[5]-[29] [Gen loss: 1.131083] [Disc loss: 0.300689] 
[5]-[30] [Gen loss: 1.861274] [Disc loss: 0.263974] 
[5]-[31] [Gen loss: 1.846710] [Disc loss: 0.286793] 
[5]-[32] [Gen loss: 1.177694] [Disc loss: 0.306943] 
[5]-[33] [Gen loss: 2.001789] [Disc loss: 0.291170] 
[5]-[34] [Gen loss: 1.151567] [Disc loss: 0.335175] 
[5]-[35] [Gen loss: 1.920141] [Disc loss: 0.381472] 
[5]-[36] [Gen loss: 0.810447] [Disc loss: 0.382713] 
[5]-[37] [Gen loss: 2.972741] [Disc loss: 0.565461] 
[5]-[38] [Gen loss: 0.441129] [Disc loss: 0.598340] 
[5]-[39] [Gen loss: 3.459382] [Disc loss: 0.604451] 
[5]-[40] [Gen loss: 0.747377] [Disc loss: 0.451167] 
[5]-[41] [Gen loss: 2.293030] [Disc loss: 0.296094] 
[5]-[42] [Gen loss: 1.616471] [Disc loss: 0.332130] 
[5]-[43] [Gen loss: 0.872462] [Disc loss: 0.390557] 
[5]-[44] [Gen loss: 2.909708] [Disc loss: 0.589184] 
[5]-[45] [Gen loss: 0.464364] [Disc loss: 0.584101] 
[5]-[46] [Gen loss: 3.324140] [Disc loss: 0.51

[5]-[186] [Gen loss: 0.553100] [Disc loss: 0.575244] 
[5]-[187] [Gen loss: 2.836359] [Disc loss: 0.601290] 
[5]-[188] [Gen loss: 0.503370] [Disc loss: 0.579832] 
[5]-[189] [Gen loss: 2.289788] [Disc loss: 0.614661] 
[5]-[190] [Gen loss: 0.552417] [Disc loss: 0.572397] 
[5]-[191] [Gen loss: 2.362985] [Disc loss: 0.527545] 
[5]-[192] [Gen loss: 0.766188] [Disc loss: 0.481744] 
[5]-[193] [Gen loss: 1.603632] [Disc loss: 0.502010] 
[5]-[194] [Gen loss: 1.051016] [Disc loss: 0.396035] 
[5]-[195] [Gen loss: 1.708316] [Disc loss: 0.422672] 
[5]-[196] [Gen loss: 0.994638] [Disc loss: 0.471866] 
[5]-[197] [Gen loss: 1.383530] [Disc loss: 0.379479] 
[5]-[198] [Gen loss: 1.524918] [Disc loss: 0.366210] 
[5]-[199] [Gen loss: 1.043386] [Disc loss: 0.434891] 
[5]-[200] [Gen loss: 1.670698] [Disc loss: 0.433836] 
[5]-[201] [Gen loss: 0.801076] [Disc loss: 0.470471] 
[5]-[202] [Gen loss: 2.107973] [Disc loss: 0.524965] 
[5]-[203] [Gen loss: 0.589363] [Disc loss: 0.524570] 
[5]-[204] [Gen loss: 2.85418